In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.create(name='azProject1',
               subscription_id='c8eb054b-91b3-425d-acbc-fe49c970c7c4',
               resource_group='UdacityProject1',
               create_resource_group=True,
               location='eastus2'
               )

exp = Experiment(workspace=ws, name="azmlProject1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Deploying StorageAccount with name azprojecstorage891f5ed74.
Deploying KeyVault with name azprojeckeyvault14ebee87.
Deploying AppInsights with name azprojecinsightsc6ce063e.
Deployed AppInsights with name azprojecinsightsc6ce063e. Took 7.26 seconds.
Deployed KeyVault with name azprojeckeyvault14ebee87. Took 25.1 seconds.
Deployed StorageAccount with name azprojecstorage891f5ed74. Took 31.67 seconds.
Deploying Workspace with name azProject1.
Deployed Workspace with name azProject1. Took 81.06 seconds.
Workspace name: azProject1
Azure region: eastus2
Subscription id: c8eb054b-91b3-425d-acbc-fe49c970c7c4
Resource group: UdacityProject1


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=2)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
from azureml.widgets import RunDetails
from azureml.core.script_run_config import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':choice(0.01,0.03,0.1,0.5,2,3,10),
                             '--max_iter':choice(50,100,150)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = ScriptRunConfig(compute_target=compute_target,
              script='train.py',
              source_directory='.')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
### YOUR CODE HERE ###
exp = Experiment(workspace=ws, name="azProject1")
run = exp.submit(hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

AttributeError: 'ScriptRunConfig' object has no attribute '_compute_target'

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metirc()
parameter_values = best_run.get_details()['runDefinition']['Arguments']
print("Run ID:",best_run.id)
print('Metrics:',best_run.metrics['accuracy'])
print('\n C:',parameter_values[0])
print('\n max_iter:',parameter_values[1])
model = best_run.register_model(model_name='hpmodel',model_path='model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
link = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=link)

In [ ]:
from train import clean_data
from sklearn.preprocessing import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.25,random_state=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    X=x,
    y=y,
    experiment_timeout_minutes=30,
    task='classifiction',
    primary_metric='accuracy',
    n_cross_validations=5,
    iterations=30)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
exp = Experiment(workspace=ws, name="azProject1")
remote_run = experiment.submit(automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
from azureml.train.automl.run import AutoMLRun
remote_run = AutoMLRun(experiment = experiment, run_id = 'HD_0b493c37-7024-4803-bf49-867685adedfa')


In [ ]:
#Exporting Model with ONNX
import onnxruntime
import onnxmltools

session = onnxruntime.InferenceSession(".")